<a href="https://colab.research.google.com/github/momo54/large_scale_data_management/blob/main/CharSets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
!wget -q "https://data.dws.informatik.uni-mannheim.de/structureddata/2021-12/quads/classspecific/schema_Mountain/part_0.gz"
!ls
lines = spark.read.text("part_0.gz").rdd.map(lambda r: r[0])


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
part_0.gz  part_0.gz.1	sample_data
2113514


[('_:nb53a24408607424384c1357880ce1bc7xb0',
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
  '<http://schema.org/Mountain>',
  '<https://peakery.com/dragontail-peak-washington/>')]

In [29]:
import re
def parseQuads(line) :
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', line)
    return parts[0], parts[1],parts[2], parts[3]
quads=lines.map(lambda x:parseQuads(x))
print(quads.count())
quads.take(1)




2113514
2113514


[('_:nb53a24408607424384c1357880ce1bc7xb0',
  'isa:<http://schema.org/Mountain>')]

In [9]:
cs=quads.map(lambda x: (x[0],x[1])).groupByKey().mapValues(list)
cs.take(5)

[('_:nb53a24408607424384c1357880ce1bc7xb0',
  ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/description>',
   '<http://schema.org/geo>',
   '<http://schema.org/image>',
   '<http://schema.org/name>']),
 ('_:nb53a24408607424384c1357880ce1bc7xb1',
  ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
   '<http://schema.org/name>',
   '<http://schema.org/value>']),
 ('_:nb53a24408607424384c1357880ce1bc7xb2',
  ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
   '<http://schema.org/name>',
   '<http://schema.org/value>']),
 ('_:nb53a24408607424384c1357880ce1bc7xb3',
  ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
   '<http://schema.org/name>',
   '<http://schema.org/value>']),
 ('_:nb53a24408607424384c1357880ce1bc7xb4',
  ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#

In [28]:
cs2=cs.map(lambda x: ('!'.join(sorted(list(dict.fromkeys(x[1])))),x[0])).groupByKey()
cs3=cs2.mapValues(lambda x: len(x))
cs3.count()

388

In [45]:
def parsePred(line) :
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', line)
    if parts[1]=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>":
      parts[1]="isa:"+parts[2]
    return parts[0], parts[1]
preds=lines.map(lambda x:parsePred(x))
print(preds.count())
preds.take(1)



2113514


46130

In [46]:
# how many mountains ??
preds.filter(lambda x: x[1].count("isa:<http://schema.org/Mountain>")>0).distinct().count()


46112

In [36]:
cspred=preds.groupByKey().mapValues(list)
cspred.take(5)

[('_:nb53a24408607424384c1357880ce1bc7xb0',
  ['isa:<http://schema.org/Mountain>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/description>',
   '<http://schema.org/geo>',
   '<http://schema.org/image>',
   '<http://schema.org/name>']),
 ('_:nb53a24408607424384c1357880ce1bc7xb1',
  ['isa:<http://schema.org/propertyValue>',
   '<http://schema.org/name>',
   '<http://schema.org/value>']),
 ('_:nb53a24408607424384c1357880ce1bc7xb2',
  ['isa:<http://schema.org/propertyValue>',
   '<http://schema.org/name>',
   '<http://schema.org/value>']),
 ('_:nb53a24408607424384c1357880ce1bc7xb3',
  ['isa:<http://schema.org/propertyValue>',
   '<http://schema.org/name>',
   '<http://schema.org/value>']),
 ('_:nb53a24408607424384c1357880ce1bc7xb4',
  ['isa:<http://schema.org/propertyValue>',
   '<http://schema.org/name>',
   '<http://schema.org/val

In [39]:
#filter on Mountains
cspred2=cspred.filter(lambda x: x[1].count("isa:<http://schema.org/Mountain>")>0)
cspred2.take(5)


[('_:nb53a24408607424384c1357880ce1bc7xb0',
  ['isa:<http://schema.org/Mountain>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/description>',
   '<http://schema.org/geo>',
   '<http://schema.org/image>',
   '<http://schema.org/name>']),
 ('_:na15b9aeffa0444219164d60919a9c91bxb0',
  ['isa:<http://schema.org/Mountain>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/geo>',
   '<http://schema.org/image>',
   '<http://schema.org/name>']),
 ('_:nb1818c388f1a48f3932fd01e4affad7cxb0',
  ['isa:<http://schema.org/Mountain>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/additionalProperty>',
   '<http://schema.org/geo>',
   '<http://schema.org/image>',
   '<http://schema.org/name>']),
 ('_:n5684a976a460406cb0a0c71eac491a61xb0',
  ['isa:<http://schema.o

In [54]:
cspred3=cspred2.map(lambda x: ('!'.join(sorted(list(dict.fromkeys(x[1])))),x[0])).groupByKey()
cspred4=cspred3.mapValues(lambda x: len(x)).sortBy(lambda x:x[1],ascending=False)

nbm=cspred4.map(lambda x:x[1]).sum()
if nbm==46112:
  print("we got all mountains")
cspred4.collect()

we got all mountains


[('<http://schema.org/additionalProperty>!<http://schema.org/geo>!<http://schema.org/image>!<http://schema.org/name>!isa:<http://schema.org/Mountain>',
  13859),
 ('<http://schema.org/additionalType>!<http://schema.org/name>!<http://schema.org/position>!<http://schema.org/url>!isa:<http://schema.org/ListItem>!isa:<http://schema.org/Mountain>',
  12801),
 ('<http://schema.org/geo>!<http://schema.org/name>!isa:<http://schema.org/Mountain>',
  5912),
 ('<http://schema.org/description>!<http://schema.org/geo>!<http://schema.org/map>!<http://schema.org/name>!isa:<http://schema.org/Mountain>',
  3843),
 ('<http://schema.org/description>!<http://schema.org/geo>!<http://schema.org/name>!<http://schema.org/url>!isa:<http://schema.org/Mountain>',
  2955),
 ('<http://schema.org/containedIn>!<http://schema.org/geo>!<http://schema.org/name>!<http://schema.org/url>!isa:<http://schema.org/Mountain>',
  1387),
 ('<http://schema.org/name>!<http://schema.org/position>!<http://schema.org/url>!isa:<http:/